In [2]:
import sys, os
sys.path.insert(0, '..')
from lib import models, graph, coarsening, utils
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from scipy import stats
import matplotlib.pyplot as plt

%matplotlib inline
flags = tf.app.flags
FLAGS = flags.FLAGS
print("finished this block")

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


finished this block


In [3]:
os.getcwd()
os.chdir('../braindata')
dd =pd.read_csv("data_3_all.csv",header=0)
import csv
with open('data_3_all.csv', 'r') as f:
    d_reader = csv.DictReader(f)
    headers = d_reader.fieldnames
print("finished this block")

finished this block


In [4]:
data=np.array(dd)



idx_IN_columns = np.append(np.array([3,4]),np.array(range(11,data.shape[1])))
X=data[:,idx_IN_columns]



X = stats.zscore(X)
y=data[:,6]

ind_num=np.isnan(y)
# print(ind_num.shape)
y_no_nan = y[~ind_num]
X_no_nan = X[~ind_num,:]

y=y_no_nan
X=X_no_nan

np.isnan(X).any()
X[np.isnan(X)] = np.median(X[~np.isnan(X)])
print("finished this block")




finished this block


/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [5]:
feature_num_all=[]
GCN_all_feature=[]
GCN_f1s_feature=[]
base_labels= []

###########################
#From Yun's code, selecting the import features
clf = ExtraTreesClassifier(n_estimators=50,random_state=0)
clf = clf.fit(X, y)
importances = clf.feature_importances_
importances
indices = np.argsort(importances)[::-1]
############################
n_features = [100,1000]
n_features.sort()


print("Finished this block")

Finished this block


In [6]:
C = max(y) + 1  # number of classes    

common = {}
common['dir_name']       = 'Chenxiao_Testing'
common['num_epochs']     = 35
common['batch_size']     = 10
common['eval_frequency'] = common['num_epochs']
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
common['filter']         = 'chebyshev5'

# Architecture.
common['F']              = [64, 32]  # Number of graph convolutional filters.
common['K']              = [10, 10]  # Polynomial orders.
common['p']              = [4, 2]    # Pooling sizes.
common['M']              = [256, 512, C]  # Output dimensionality of fully connected layers.

# Optimization.
common['regularization'] = 5e-4
common['dropout']        = 0.9
common['learning_rate']  = 5e-4
common['decay_rate']     = 0.95
common['momentum']       = 0.9
#params['decay_steps']    = n_train / params['batch_size']

model_perf = utils.model_perf()    

sep = '*' * 100
print("finished this block")

finished this block


In [ ]:
""""

for i in n_features:
    print('\nWorking on number of feature selected as {} {}\n'.format(i, sep))
    index=indices[0:i]
    features=X[:,index]
    feature_num=features.shape[1]

    GCN_all_accs = []
    GCN_all_scores = []
    GCN_f1s = []
    base_labels=[]

###############################################################################
###############################################################################
#This part is from the NIPS code, generating training, validation and traing set

        
    for runs in range(10):        
        print("\n RUN: {} {} \n".format(runs, sep))

        GCN_accuracies = []
        GCN_scores = []
        strat_labels = []
        
        for j in range(skf.get_n_splits(features, y)):   
            print("\n Fold: {} {} \n".format(i, sep))
            
            train_data = np.empty([0,features.shape[1]])
            train_labels = [] 
            val_ix = j
            if val_ix == max(bins):
                test_ix = 0
                bins.remove(val_ix)
                bins.remove(test_ix)

            else:
                test_ix = val_ix + 1
                bins.remove(test_ix)
                bins.remove(val_ix)

            val_data, val_labels = features[bin_ixs[val_ix]], y[bin_ixs[val_ix]]
            test_data, test_labels = features[bin_ixs[test_ix]], y[bin_ixs[test_ix]]
        
            for b in bins:
                train_data = np.concatenate((train_data, features[bin_ixs[b]]))
                train_labels = np.concatenate((train_labels, y[bin_ixs[b]]))

###################################################
#Chenxiao: generating permuational matrix
            dist, idx = graph.distance_scipy_spatial(train_data.transpose(), k=10, metric='euclidean')
            A = graph.adjacency(dist, idx).astype(np.float32)

            assert A.shape == (d, d)
            print('d = |V| = {}, k|V| < |E| = {}'.format(d, A.nnz))
            plt.spy(A, markersize=2, color='black')     
        
            graphs, perm = coarsening.coarsen(A, levels=8, self_connections=False)
        
            train_data = coarsening.perm_data(train_data, perm)
            val_data = coarsening.perm_data(val_data, perm)
            test_data = coarsening.perm_data(test_data, perm)

            L = [graph.laplacian(A, normalized=True) for A in graphs]
 #       graph.plot_spectrum(L)    
 ###################################################        
            name = 'CGCNN'
            params = common.copy()
            params['dir_name'] += name             
            params['decay_steps'] = len(train_labels) / common['batch_size']
        
            print('begin working!!!!!!!!')
            model_perf.test(models.cgcnn(L, **params), name, params, 
                            train_data, train_labels, val_data, val_labels, test_data, test_labels, val_ix)
        
            scores, y_labels, test_accuracy = model_perf.show()   
            
            strat_labels=np.append(strat_labels, test_labels)
            
            #f1 calculation
            log_f1=f1_score(test_labels, y_labels)            

            GCN_accuracies = np.append(GCN_accuracies, test_accuracy)
            GCN_scores=np.append(GCN_scores, scores)
            GCN_f1s=np.append(GCN_f1s, log_f1)
            
            GCN_all_scores=np.append(GCN_all_scores, GCN_scores) 
            GCN_fold_avg=np.mean(GCN_accuracies)
            GCN_all_accs=np.append(GCN_all_accs, GCN_fold_avg)
            
        feature_num_all=np.append(feature_num_all, feature_num)
        GCN_all_feature=np.append(GCN_all_feature, np.mean(GCN_all_accs))
        GCN_f1s_feature=np.append(GCN_f1s_feature, np.mean(GCN_f1s)
        base_labels=np.append(base_labels, np.mean(strat_labels))
""""

In [ ]:
##################################
##################################
#This is from Yun's framework of K-fold cross validation
##################################
##################################


###########################
#From Yun's code, selecting the import features
clf = ExtraTreesClassifier(n_estimators=250,random_state=0)
clf = clf.fit(X, y)
importances = clf.feature_importances_
importances
indices = np.argsort(importances)[::-1]
############################


number_of_features=500
index=indices[0:number_of_features]
features=X[:,index]
feature_num=features.shape[1]

GCN_all_accs_one_fetur  = []
GCN_all_scores_one_fetur = []
GCN_all_f1s_one_fetur = []

LR_all_accs_one_fetur  = []
LR_all_f1s_one_fetur = []

SVM_all_accs_one_fetur  = []
SVM_all_f1s_one_fetur = []
base_labels=[]





###############################################################################
###############################################################################
#This part is from the NIPS code, generating training, validation and traing set

for runs in range(5):        
    counter=0
    print("\n RUN: {} {} \n".format(runs, sep))
    strat_labels = []

    GCN_accuracies = []
    GCN_scores = []
    GCN_f1s=[]

    LR_accuracies = []
    LR_f1s=[]    
    
    SVM_accuracies = []
    SVM_f1s=[]    
    
    t=time.process_time()
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=int(t))
#    skf = StratifiedKFold(n_splits=10, shuffle=True)

    for train_index, test_index in skf.split(features, y):
        print("\n Fold: {} {} \n".format(counter, sep))        
        train_data, test_data = features[train_index], features[test_index]
        train_labels, test_labels = y[train_index], y[test_index]
        val_data=test_data
        val_labels=test_labels

###################################################
#Chenxiao: generating permuational matrix
        dist, idx = graph.distance_scipy_spatial(train_data.transpose(), k=10, metric='euclidean')
        A = graph.adjacency(dist, idx).astype(np.float32)

        assert A.shape == (features.shape[1], features.shape[1])
        print('d = |V| = {}, k|V| < |E| = {}'.format(features.shape[1], A.nnz))
        plt.spy(A, markersize=2, color='black')     
        
        graphs, perm = coarsening.coarsen(A, levels=6, self_connections=False)
        
        train_data = coarsening.perm_data(train_data, perm)
        val_data = coarsening.perm_data(val_data, perm)
        test_data = coarsening.perm_data(test_data, perm)

        L = [graph.laplacian(A, normalized=True) for A in graphs]
 #       graph.plot_spectrum(L)    
 ###################################################        
    
        name = 'CGCNN'
        params = common.copy()
        params['dir_name'] = params['dir_name'] + name + '_Run' + str(runs) + '_Skf' + str(counter)             
        params['decay_steps'] = len(train_labels) / common['batch_size']
        
        print('begin working!!!!!!!!')
        model_perf.test(models.cgcnn(L, **params), name, params, 
                        train_data, train_labels, val_data, val_labels, test_data, test_labels, test_index)
        
        scores, f1, y_labels, test_accuracy = model_perf.show()   
            
        strat_labels=np.append(strat_labels, test_labels)
            
####################################################   
#Chenxiao: add Yun's code for logistric regression and SVM, for comparision with GCN case by case
        logistic = linear_model.LogisticRegression(C=1e5)
        rbf = svm.SVC(C=10,kernel='linear',gamma=0.01)
#######################
#logistic regression:

        log=logistic.fit(train_data, train_labels)
    
        #log_prob_val=log.decision_function(val_data)
        #log_prob_test=log.decision_funtion(test_data)
        
        log_acc_val=log.score(val_data, val_labels)
        log_acc_test=log.score(test_data, test_labels)    
        
        ##f1 calculation
        y_pred_val=logistic.predict(val_data)
        y_pred_test=logistic.predict(test_data)
        
        log_f1_val=f1_score(val_labels, y_pred_val)
        log_f1_test=f1_score(test_labels, y_pred_test)
        
        
        print('Logistic Regression Accuracy for Validation: %f' % log_acc_val)
        print('Logistic Regression Accuracy for Test: %f' % log_acc_test)
        print('Logistic Regression F1 for Validation: %f' % log_f1_val)
        print('Logistic Regression F1 for Test: %f' % log_f1_test)
        
        
#SVM:
        rbf=rbf.fit(train_data, train_labels)
    
        #svm_prob_val=rbf.decision_function(val_data)
        #svm_prob_test=rbf.decision_funtion(test_data)
        
        svm_acc_val=rbf.score(val_data, val_labels)
        svm_acc_test=rbf.score(test_data, test_labels)    
        
        #f1 calculation
        y_pred_val=rbf.predict(val_data)
        y_pred_test=rbf.predict(test_data)
        
        svm_f1_val=f1_score(val_labels, y_pred_val)
        svm_f1_test=f1_score(test_labels, y_pred_test)
               
        print('SVM Accuracy for Validation: %f' % svm_acc_val)
        print('SVM Accuracy for Test: %f' % svm_acc_test)
        print('SVM F1 for Validation: %f' % svm_f1_val)
        print('SVM F1 for Test: %f' % svm_f1_test)        
                
####################################################        
        
        
        ##f1 calculation
        #log_f1=f1_score(test_labels, y_labels)            

        GCN_accuracies = np.append(GCN_accuracies, test_accuracy)
        GCN_scores=np.append(GCN_scores, scores)
        GCN_f1s=np.append(GCN_f1s, f1)
        
        LR_accuracies = np.append(LR_accuracies, log_acc_test)
        LR_f1s=np.append(LR_f1s, log_f1_test)        
        
        SVM_accuracies = np.append(SVM_accuracies, svm_acc_test)
        SVM_f1s=np.append(SVM_f1s, svm_f1_test) 
        
        counter=counter+1
   
    base_labels=np.append(base_labels, strat_labels)        
    GCN_all_scores_one_fetur=np.append(GCN_all_scores_one_fetur, GCN_scores) 
    GCN_all_accs_one_fetur=np.append(GCN_all_accs_one_fetur, np.mean(GCN_accuracies))
    GCN_all_f1s_one_fetur =np.append(GCN_all_f1s_one_fetur, np.mean(GCN_f1s))
    
    LR_all_accs_one_fetur=np.append(LR_all_accs_one_fetur, np.mean(LR_accuracies))
    LR_all_f1s_one_fetur =np.append(LR_all_f1s_one_fetur, np.mean(LR_f1s))    
    
    SVM_all_accs_one_fetur=np.append(SVM_all_accs_one_fetur, np.mean(SVM_accuracies))
    SVM_all_f1s_one_fetur =np.append(SVM_all_f1s_one_fetur, np.mean(SVM_f1s))       
    
    print("GCN###############################################################")
    print("Runs Avg Accuracies: {}".format(np.mean(GCN_accuracies)))
    print("Standard Deviation: {}".format(np.std(GCN_accuracies)))
    print("Runs Avg F1: {}".format(np.mean(GCN_f1s)))
    print("Standard Deviation: {}".format(np.std(GCN_f1s)))

    print("LR###############################################################")
    print("Runs Avg Accuracies: {}".format(np.mean(LR_accuracies)))
    print("Standard Deviation: {}".format(np.std(LR_accuracies)))
    print("Runs Avg F1: {}".format(np.mean(LR_f1s)))
    print("Standard Deviation: {}".format(np.std(LR_f1s)))

    print("SVM###############################################################")
    print("Runs Avg Accuracies: {}".format(np.mean(SVM_accuracies)))
    print("Standard Deviation: {}".format(np.std(SVM_accuracies)))
    print("Runs Avg F1: {}".format(np.mean(SVM_f1s)))
    print("Standard Deviation: {}".format(np.std(SVM_f1s)))    


 RUN: 0 **************************************************************************************************** 


 Fold: 0 **************************************************************************************************** 

d = |V| = 500, k|V| < |E| = 8376
Layer 0: M_0 = |V| = 576 nodes (76 added),|E| = 4188 edges
Layer 1: M_1 = |V| = 288 nodes (30 added),|E| = 2791 edges
Layer 2: M_2 = |V| = 144 nodes (14 added),|E| = 1704 edges
Layer 3: M_3 = |V| = 72 nodes (6 added),|E| = 867 edges
Layer 4: M_4 = |V| = 36 nodes (2 added),|E| = 357 edges
Layer 5: M_5 = |V| = 18 nodes (1 added),|E| = 118 edges
Layer 6: M_6 = |V| = 9 nodes (0 added),|E| = 35 edges
begin working!!!!!!!!


In [ ]:
print("GCN###############################################################")
print("Runs Avg Accuracies: {}".format(np.mean(GCN_all_accs_one_fetur)))
print("Standard Deviation: {}".format(np.std(GCN_all_accs_one_fetur)))
print("Runs Avg F1: {}".format(np.mean(GCN_all_f1s_one_fetur)))
print("Standard Deviation: {}".format(np.std(GCN_all_f1s_one_fetur)))

print("LR###############################################################")
print("Runs Avg Accuracies: {}".format(np.mean(LR_all_accs_one_fetur)))
print("Standard Deviation: {}".format(np.std(LR_all_accs_one_fetur)))
print("Runs Avg F1: {}".format(np.mean(LR_all_f1s_one_fetur)))
print("Standard Deviation: {}".format(np.std(LR_all_f1s_one_fetur)))

print("SVM###############################################################")
print("Runs Avg Accuracies: {}".format(np.mean(SVM_all_accs_one_fetur)))
print("Standard Deviation: {}".format(np.std(SVM_all_accs_one_fetur)))
print("Runs Avg F1: {}".format(np.mean(SVM_all_f1s_one_fetur)))
print("Standard Deviation: {}".format(np.std(SVM_all_f1s_one_fetur)))

In [ ]:
from sklearn.metrics import roc_curve, auc,f1_score
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()




fpr['GCN'], tpr['GCN'], _ = roc_curve(base_labels, GCN_all_scores_one_fetur)
roc_auc['GCN'] = auc(fpr['GCN'], tpr['GCN'])


plt.figure()
plt.plot(fpr['GCN'], tpr['GCN'], lw=5, label='GCN (area = %0.2f)' % roc_auc['GCN'] )
plt.plot([0, 1], [0, 1], color='navy', lw=5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('MCIvsSMI')
plt.legend(loc="lower right")
#plt.savefig('10x_Combined_ROC.eps')
#plt.savefig('ROC_MCIVsNormal_connectome.eps')
#plt.savefig('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/papers/ISMRM/Figures/ROC_MCIvsSMI_connectome.pdf')
plt.show()